# Q1_Answer

运行 OneDimDiffusion.zip 中的程序，体会、理解和掌握有限差分和有限体积法的求解 1D 热传导方程的基本步骤

* 第一步：初始化相关参数

In [1]:
node_num = 60  # 定义网格数量为 60
tlist_new = [36.8] * node_num  # 记录新温度
tlist_old = [36.8] * node_num  # 记录旧温度
tlist_new[0] = 100  # 左侧边界温度

max_loops = 300000  # 迭代次数
step_before_1w = 1000  # 迭代次数为 1w 前（含）步长
step_after_1w = 10000  # 迭代次数为 1w 后步长
loop_count = 0  # 迭代保存计数器

In [2]:
delta_x = 0.5
delta_t = 0.1
alpha = 0.001
alpha /= (delta_x * delta_x / delta_t)

* 第二步：定义有限差分算法

In [3]:
def thermal_conductivity(T_old):
    for i in range(1, node_num-1):
        tlist_new[i] = tlist_old[i] + alpha * \
            (tlist_old[i + 1] - 2 * tlist_old[i] + tlist_old[i - 1])
    tlist_new[node_num - 1] = tlist_new[node_num - 2]
    return tlist_new

* 第三步：启动循环

In [4]:
import plotly_express as px  # 导入绘图库

In [5]:
thermal_data = []
for loops in range(max_loops+1):
    tlist_old = thermal_conductivity(tlist_old)
    if loops % step_before_1w == 0:
        if loops <= 10000 or loops % step_after_1w == 0:
            thermal_data.extend(tlist_old[::2])  # 隔一采样
            loop_count += 1

* 第四步：数据处理

In [6]:
from pandas import DataFrame

In [9]:
loop = []
node_num_half = node_num // 2
for i in range(loop_count):
    if i <= 10:
        loop.extend([i*step_before_1w]*node_num_half)
    else:
        loop.extend([(i-9)*step_after_1w]*node_num_half)

data = {"loop": loop, "x": [i*2 for i in range(
    node_num_half)]*(loop_count), "temperature": thermal_data}

data_frame = DataFrame(data)

* 第五步：绘图

In [10]:
px.line(data_frame, x="x", y="temperature", animation_frame="loop")